## Q1

In [ ]:
import numpy as np
import cv2

def generateSimpleGrayscaleImage(width, height):
    # Start with a white background
    image = np.full((height, width), 255, dtype=np.uint8)

    # Draw first object with pixel value 128
    cv2.rectangle(image, (100, 150), (200, 250), color=(128,), thickness=-1)

    # Draw second object with pixel value 200
    cv2.circle(image, (300, 180), radius=30, color=(160,), thickness=-1)

    return image

def addGaussianNoise(image, mean=0, stddev=15):
    # Convert to float32 for noise addition
    noise = np.random.normal(mean, stddev, image.shape).astype(np.float32)
    image_float = image.astype(np.float32)
    noisy = np.clip(image_float + noise, 0, 255).astype(np.uint8)
    return noisy

# Step 1: Create image with 3 distinct grayscale pixel values
base_image = generateSimpleGrayscaleImage(600, 600)
cv2.imshow("Original 3-Pixel Image", base_image)
cv2.imwrite("original_grayscale_3pixel.png", base_image)

# Step 2: Add Gaussian noise
noisy_image = addGaussianNoise(base_image)
cv2.imshow("Noisy Image", noisy_image)
cv2.imwrite("noisy_image.png", noisy_image)

# Step 3: Apply Otsu's thresholding
_, otsu_result = cv2.threshold(noisy_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
cv2.imshow("Otsu's Thresholding Result", otsu_result)
cv2.imwrite("otsu_result.png", otsu_result)

cv2.waitKey(0)
cv2.destroyAllWindows()


## Q2


In [ ]:
import cv2

# Load the image in color (do NOT convert to grayscale)
image = cv2.imread('Q2/image.jpg')

# Make a copy of the original image
image_with_seeds = image.copy()

# Define seed points (x, y)
seed_points = [(200, 100), (500, 100), (700, 100), (500, 500), (500, 300)]

# Mark seed points on the image (draw small red circles)
for (x, y) in seed_points:
    cv2.circle(image_with_seeds, (x, y), radius=5, color=(0, 0, 255), thickness=-1)  # Red color in BGR

# Display original and marked image
cv2.imshow("Original Image", image)
cv2.imshow("Image with Seed Points", image_with_seeds)
cv2.imwrite("image_with_seed_points.png", image_with_seeds)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

def show_segmentation(mask):
    cv2.imshow('Segmentation Process', mask)
    cv2.waitKey(1)  # short delay to update display

def region_growing(image, seed_points, threshold_range):
    # Initialize mask: 0 = background/unvisited, 255 = segmented/visited
    mask = np.zeros_like(image, dtype=np.uint8)
    queue = list(seed_points)
    iteration = 0
    while queue:
        iteration += 1
        current_point = queue.pop(0)
        x, y = current_point
        current_value = image[y, x]
        # Mark current point as segmented
        mask[y, x] = 255
        if iteration % 10 == 0:
            show_segmentation(mask)
        # Check 8 neighbors
        for i in range(-1, 2):
            for j in range(-1, 2):
                if i == 0 and j == 0:
                    continue
                nx, ny = x + i, y + j
                # Check boundaries
                if 0 <= nx < image.shape[1] and 0 <= ny < image.shape[0]:
                    neighbor_value = image[ny, nx]
                    
                    # If neighbor pixel within threshold and not yet segmented
                    if abs(int(neighbor_value) - int(current_value)) <= threshold_range and mask[ny, nx] == 0:
                        queue.append((nx, ny))
                        mask[ny, nx] = 255
                        
    return mask

# Load image in grayscale
image = cv2.imread('Q2/image.jpg', cv2.IMREAD_GRAYSCALE)

# Seed points (x, y)
seed_points = [(200, 100), (500, 100), (700, 100), (500, 500), (500, 300)]

# Threshold for similarity
threshold_range = 10

# Run region growing segmentation
segmented_image = region_growing(image, seed_points, threshold_range)

cv2.destroyAllWindows()
cv2.imshow('Segmented Image', segmented_image)
cv2.imwrite('segmented_image.png', segmented_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
